In [ ]:
import numpy as np
import cv2
import argparse
from collections import deque


cap=cv2.VideoCapture(0)

pts = deque(maxlen=10000)

Lower_green = np.array([100,50,50]) #0,0,50(RED) #10,0,255 #100,50,50(Blue)
Upper_green = np.array([130,255,255])#15,255,255 #   #130,255,255
while True:
	ret, img=cap.read()
	hsv=cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
	kernel=np.ones((5,5),np.uint8)
	mask=cv2.inRange(hsv,Lower_green,Upper_green)
	mask = cv2.erode(mask, kernel, iterations=2)
	mask=cv2.morphologyEx(mask,cv2.MORPH_OPEN,kernel)
	#mask=cv2.morphologyEx(mask,cv2.MORPH_CLOSE,kernel)
	mask = cv2.dilate(mask, kernel, iterations=1)
	res=cv2.bitwise_and(img,img,mask=mask)
	cnts,heir=cv2.findContours(mask.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[-2:]
	center = None
 
	if len(cnts) > 0:
		c = max(cnts, key=cv2.contourArea)
		((x, y), radius) = cv2.minEnclosingCircle(c)
		M = cv2.moments(c)
		center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
		print(f"Contour's center: {center}")

	
 
		if radius > 5:
			if center!=None and center[0] > 1000 and center[0]<1200 and center[1]>100 and center[1]<500:
				cv2.circle(img, (int(x), int(y)), int(radius),(0, 0, 255), 2)
				cv2.circle(img, center, 5, (0, 255, 255), -1)
				color = (0,128,0)

			else:
				cv2.circle(img, (int(x), int(y)), int(radius),(0, 255, 255), 2)
				cv2.circle(img, center, 5, (0, 0, 255), -1) 
				color = (0,0,255)
				cv2.circle(mask, (int(x), int(y)), int(radius),(130,50,50), 2)
				cv2.circle(mask, center, 5, (130,50,50), -1)
		
	pts.appendleft(center)
	for i in range (1,len(pts)):
		if pts[i-1]is None or pts[i] is None:
			continue
		#thick = int(np.sqrt(len(pts) / float(i + 1)) * 2.5)
		thick =10
		cv2.line(img, pts[i-1],pts[i],color,thick)
		cv2.line(mask, pts[i-1],pts[i],(130,50,50),thick)
		
	flipped = cv2.flip(img,1)
	flipped2 = cv2.flip(mask,1)
	window_name="Projector"
	w_n = "Hello"
	cv2.namedWindow(window_name, cv2.WND_PROP_FULLSCREEN)
	cv2.namedWindow(w_n, cv2.WND_PROP_FULLSCREEN)
	#cv2.imshow(window_name, flipped)
	cv2.imshow(window_name, flipped)
	cv2.imshow(w_n,flipped2)
	#cv2.imshow("res",res)
	
	
	k=cv2.waitKey(30) & 0xFF
	if k==32:
		break
# cleanup the camera and close any open windows
cap.release()
cv2.destroyAllWindows()
